# Train test split KVLCC2 HSVA
<cite id="hzn48"><a href="#zotero|7505983/U2L55C9Q">(Luo et al., 2016)</a></cite>

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from src.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib
matplotlib.rcParams["figure.figsize"] = (10,10)
from src.symbols import *

# Read configs:
conf_path = os.path.join("../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmm_names = global_variables["vmms"]

from wPCC_pipeline.pipelines.motion_regression.nodes import predict_force, fit_motions, create_model_from_motion_regression
from wPCC_pipeline.pipelines.prediction.nodes import simulate_euler
from src.models.vmm import VMM
from src.parameters import df_parameters

In [ ]:
ship="kvlcc2_hsva"
vmm_name = "vmm_martins_simple"
#vmm_name = "vmm_abkowitz"
vmm = catalog.load(vmm_name)

ship_data = catalog.load(f"{ship}.ship_data")

train = "HSVA_CPMC_KVLCC2_Z_25_05"
df_train = catalog.load(f'{ship}.updated.{train}.data_ek_smooth')

test = "HSVA_CPMC_KVLCC2_Z_35_05"
df_test = catalog.load(f'{ship}.updated.{test}.data_ek_smooth')

#delta_start = np.deg2rad(10.5)
#
#for data in [df_train, df_test]:
#    index_start = (data['delta'].diff().abs() > 0).idxmax()
#    start = data.loc[0:index_start] 
#    start['delta'] = delta_start

#index_start = (df_train['delta'].diff().abs() > 0).idxmax()
#df_train = df_train.loc[index_start:].copy()
#df_train.index = df_train.index - df_train.index[0]

index_start = (df_test['delta'].diff().abs() > 0).idxmax()
df_test = df_test.loc[index_start:].copy()
df_test.index = df_test.index - df_test.index[0]


added_masses = catalog.load(f"{ship}.added_masses")
exclude_parameters = catalog.load(f"params:{ship}.motion_regression.exclude_parameters")


In [ ]:
data = df_train
data_with_force = predict_force(data=data, added_masses=added_masses, ship_parameters=ship_data, vmm=vmm)


In [ ]:
vmm.X_eq

In [ ]:
vmm.Y_eq

In [ ]:
vmm.N_eq

In [ ]:
regression, parameters = fit_motions(data=data_with_force, added_masses=added_masses, ship_data=ship_data, vmm=vmm, exclude_parameters=exclude_parameters)

In [ ]:
model = create_model_from_motion_regression(regression=regression)

In [ ]:
p = df_parameters["symbol"]


vmm_extras = []


X_eq2=sp.Eq(vmm.X_eq.lhs, vmm.X_eq.rhs+ p.Xuu*u**2)
Y_eq2 = vmm.Y_eq.copy()
N_eq2 = vmm.N_eq.copy()
vmm2 = manoeuvring model(X_eq=X_eq2, Y_eq=Y_eq2, N_eq=N_eq2)
vmm_extras.append(vmm2)


X_eq2=sp.Eq(vmm.X_eq.lhs, vmm.X_eq.rhs+ p.Xuu*u**2)
Y_eq2 = vmm.Y_eq.subs([(p.Yu,0),
                       (p.Ythrust,0)])
N_eq2 = vmm.N_eq.subs([(p.Nu,0),
                      (p.Nthrust,0),
                     ])

vmm2 = manoeuvring model(X_eq=X_eq2, Y_eq=Y_eq2, N_eq=N_eq2)
vmm_extras.append(vmm2)

X_eq2=sp.Eq(vmm.X_eq.lhs, vmm.X_eq.rhs+ p.Xuu*u**2)
Y_eq2 = sp.Eq(vmm.Y_eq.lhs, vmm.Y_eq.rhs +  p.Yvvdelta*v**2*delta)
N_eq2 = sp.Eq(vmm.N_eq.lhs, vmm.N_eq.rhs+ p.Nvvdelta*v**2*delta)

vmm2 = manoeuvring model(X_eq=X_eq2, Y_eq=Y_eq2, N_eq=N_eq2)
vmm_extras.append(vmm2)



In [ ]:
extra_regressions = []
extra_models = []

for vmm2 in vmm_extras:

    regression2, parameters2 = fit_motions(data=data_with_force, added_masses=added_masses, ship_data=ship_data, vmm=vmm2, exclude_parameters=exclude_parameters)
    model2 = create_model_from_motion_regression(regression=regression2)
    extra_regressions.append(regression2)
    extra_models.append(model2)
    
    

In [ ]:
model2.parameters['Ydelta']*model2.ship_parameters_prime['x_r']

In [ ]:
model2.parameters['Ndelta']

In [ ]:
df_ = df_test.copy()
#df_.iloc[0]['r'] = 0
#df_.iloc[0]['r'] = -0.03
#df_['delta']+=np.deg2rad(1.5)
    
result = model.simulate(df_=df_)

df_result = result.result.copy()


df_result['psi_deg'] = np.rad2deg(df_result['psi'])

df_test['psi_deg'] = np.rad2deg(df_test['psi'])
df_result['delta_deg'] = np.rad2deg(df_result['delta'])

df_test['delta_deg'] = np.rad2deg(df_test['delta'])

dataframes = {
    'Experiment' : df_test,
    'Prediction' : df_result,
}

styles = {
    'Experiment' : {'style':'r--'},
    'Prediction' : {'style':'k-'},
}




In [ ]:
for i,model_ in enumerate(extra_models):
    result2 = model_.simulate(df_=df_)
    df_result2 = result2.result.copy()
    df_result2['psi_deg'] = np.rad2deg(df_result2['psi'])
    df_result2['delta_deg'] = np.rad2deg(df_result2['delta'])
    dataframes[f"{i}"] = df_result2

In [ ]:
vmm_extras[1].Y_eq

In [ ]:
vmm_extras[1].N_eq

In [ ]:
np.rad2deg(df_test['delta'].mean())

In [ ]:
track_plots(dataframes, lpp=ship_data['L'], beam=ship_data['B'],  styles=styles);

In [ ]:
plot(dataframes=dataframes, keys=['delta_deg','psi_deg','y0','u','v','r'], ncols=1, styles=styles, time_window=[0,2]);
plot(dataframes=dataframes, keys=['delta_deg','psi_deg','y0','u','v','r'], ncols=1, styles=styles);

In [ ]:
vmm_extras[1].Y_eq

## Monte Carlo

In [ ]:
from scipy.linalg import block_diag
from scipy.stats import norm, multivariate_normal
import tqdm
import sys
from wPCC_pipeline.pipelines.prediction.nodes import simulate_euler

In [ ]:
ek = catalog.load(f"{ship}.{vmm_name}.ek")

In [ ]:
def monte_carlo(data_smooth, df_parameter_variation, model, ek):

    dataframes = {}
    with tqdm.tqdm(total=len(df_parameter_variation), file=sys.stdout) as pbar:
        for index, parameters_ in df_parameter_variation.iterrows():
            model_ = model.copy()
            model_.parameters.update(parameters_)
            
            df_ = simulate_euler(data=data_smooth, model=model_,ek=ek, solver='Radau') 
            dataframes[index] = df_
        
            pbar.update(1)
        
    return dataframes

In [ ]:
#regression = regression_abkowitz
means = pd.concat([regression.model_X.params,regression.model_Y.params,regression.model_N.params])
covs = [regression.model_X.cov_HC0,regression.model_Y.cov_HC0,regression.model_N.cov_HC0]

std = regression.model_X.bse.copy()
std=std.append(regression.model_Y.bse)
std=std.append(regression.model_N.bse)

cov = block_diag(*covs)

rv = multivariate_normal(mean=means.values, cov=cov, allow_singular=True)
np.random.seed(42)
N_=20
df_parameter_variation = pd.DataFrame(data=rv.rvs(N_), columns=means.index)

In [ ]:
dataframes = monte_carlo(df_test, df_parameter_variation, model=model, ek=ek)

In [ ]:
dataframes['model test'] = df_test
#dataframes['manoeuvring model all'] = df_result

styles={'model test':{'style':'k-','lw':2},
       'manoeuvring model all':{'style':'r-','lw':2},
        
       }

for index, parameters_ in df_parameter_variation.iterrows():
    styles[index] = {'style':'b-', 
                     'alpha':0.1,
                     'label':'_Hidden label'}

In [ ]:
fig,ax=plt.subplots()
track_plots(dataframes, lpp=ship_data['L'], beam=ship_data['B'], plot_boats=True, flip=True, N=2, styles=styles, ax=ax)

In [ ]:
plot(dataframes, keys=['delta_deg','psi_deg','y0','u','v','r'], ncols=1, styles=styles);

In [ ]:
model.parameters

In [ ]:
regression.model_Y.summary()

In [ ]:
regression.model_Y.pvalues